## Importing Libraries and Data

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/scripts'))
import matplotlib.pyplot as plt
import numpy as np
import eda


In [2]:
df_trip = pd.read_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/nb.csv')
df_locations = pd.read_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/driver_locations_during_request.csv')

# EDA Analysis

### Overview

In [3]:
df_trip.head(3)

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23


In [75]:
df_locations.head(3)

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN


Preparing df_trip data to merge

In [4]:
df_org = df_trip['Trip Origin'].str.split(',', expand=True)
df_org.columns = ['lat_org', 'lon_org']


# Combining the new columns with the original DataFrame
df_trip = pd.concat([df_trip, df_org], axis=1)
df_trip = df_trip.drop('Trip Origin', axis=1)

df_des = df_trip['Trip Destination'].str.split(',', expand=True)
df_des.columns = ['lat_des', 'lon_des']


# # Combining the new columns with the original DataFrame
df_trip = pd.concat([df_trip, df_des], axis=1)
df_trip = df_trip.drop('Trip Destination', axis=1)


df_start = df_trip['Trip Start Time'].str.split(' ', expand=True)
df_start.columns = ['Start_date', 'Start_time']


# Combining the new columns with the original DataFrame
df_trip = pd.concat([df_trip, df_start], axis=1)
df_trip = df_trip.drop('Trip Start Time', axis=1)

df_end = df_trip['Trip End Time'].str.split(' ', expand=True)
df_end.columns = ['End_date', 'End_time']

# Combining the new columns with the original DataFrame
df_trip = pd.concat([df_trip, df_end], axis=1)
df_trip = df_trip.drop('Trip End Time', axis=1)
df_trip.head(3)

,Trip ID,lat_org,lon_org,lat_des,lon_des,Start_date,Start_time,End_date,End_time
0,391996,6.508813001668548,3.37740316890347,6.650969799999999,3.3450307,2021-07-01,07:28:04,2021-07-01,07:29:37
1,391997,6.4316714,3.4555375,6.4280814653326,3.4721885847586,2021-07-01,06:38:04,2021-07-01,07:07:28
2,391998,6.631679399999999,3.3388976,6.508324099999999,3.3590397,2021-07-01,06:21:02,2021-07-01,07:02:23


Preparing df_locations data to prepare for merging

In [5]:
df_trip.rename(columns={'Trip ID': 'order_id'}, inplace=True)
df_locations.rename(columns={'lat': 'lat_org'}, inplace=True)
df_locations.rename(columns={'lng': 'lon_org'}, inplace=True)
df_trip.rename(columns={'End_date': 'date'}, inplace=True)

# adding features to df_location
lat_lon = df_trip[['order_id','lat_des','lon_des','date']]
df_locations = df_locations.merge(lat_lon, on='order_id', how='left')
df_locations.drop(columns=['driver_id'], inplace=True)
df_locations.drop(columns=['created_at'], inplace=True)
df_locations.drop(columns=['updated_at'], inplace=True)
df_locations.drop(columns=['id'], inplace=True)
df_locations.head(5)

,order_id,driver_action,lat_org,lon_org,lat_des,lon_des,date
0,392001,accepted,6.602207,3.270465,6.4501069,3.3916154,2021-07-01
1,392001,rejected,6.592097,3.287445,6.4501069,3.3916154,2021-07-01
2,392001,rejected,6.596133,3.281784,6.4501069,3.3916154,2021-07-01
3,392001,rejected,6.596142,3.280526,6.4501069,3.3916154,2021-07-01
4,392001,rejected,6.609232,3.288800,6.4501069,3.3916154,2021-07-01


In [6]:
df_trip.drop(columns=['Start_time'], inplace=True)
df_trip.drop(columns=['Start_date'], inplace=True)
df_trip.drop(columns=['End_time'], inplace=True)
df_trip['driver_action']= 'accepted'
df_trip.head()

,order_id,lat_org,lon_org,lat_des,lon_des,date,driver_action
0,391996,6.508813001668548,3.37740316890347,6.650969799999999,3.3450307,2021-07-01,accepted
1,391997,6.4316714,3.4555375,6.4280814653326,3.4721885847586,2021-07-01,accepted
2,391998,6.631679399999999,3.3388976,6.508324099999999,3.3590397,2021-07-01,accepted
3,391999,6.572757200000001,3.3677082,6.584881099999999,3.3614073,2021-07-01,accepted
4,392001,6.6010417,3.2766339,6.4501069,3.3916154,2021-07-01,accepted


merging the data

In [7]:

df_concatenated = pd.concat([df_trip, df_locations], axis=0, ignore_index=True)
df_concatenated.head(5)

,order_id,lat_org,lon_org,lat_des,lon_des,date,driver_action
0,391996,6.508813001668548,3.37740316890347,6.650969799999999,3.3450307,2021-07-01,accepted
1,391997,6.4316714,3.4555375,6.4280814653326,3.4721885847586,2021-07-01,accepted
2,391998,6.631679399999999,3.3388976,6.508324099999999,3.3590397,2021-07-01,accepted
3,391999,6.572757200000001,3.3677082,6.584881099999999,3.3614073,2021-07-01,accepted
4,392001,6.6010417,3.2766339,6.4501069,3.3916154,2021-07-01,accepted


## Dealing with Missing Values

In [85]:
df_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093760 entries, 0 to 2093759
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   order_id       int64 
 1   lat_org        object
 2   lon_org        object
 3   lat_des        object
 4   lon_des        object
 5   date           object
 6   driver_action  object
dtypes: int64(1), object(6)
memory usage: 111.8+ MB


In [86]:
df_concatenated.isnull().sum()

order_id         0
lat_org          0
lon_org          0
lat_des          0
lon_des          0
date             1
driver_action    0
dtype: int64

### Dropping null values based on threshold

In [8]:
df_concatenated = eda.drop_missing_values(df_concatenated, threshold=0.05)

In [89]:
df_concatenated.isnull().sum()

order_id         0
lat_org          0
lon_org          0
lat_des          0
lon_des          0
date             0
driver_action    0
dtype: int64

## Dealing with outliers

In [97]:
df_concatenated.describe()

,order_id
count,2.093759e+06
mean,5.655788e+05
std,3.049954e+05
min,3.919960e+05
25%,4.118390e+05
50%,4.260790e+05
75%,4.479770e+05
max,1.637709e+06


In [ ]:
columns_to_check = ['lat_org','lon_org','lat_des', 'lon_des']
df_concatenated[columns_to_check] = df_concatenated[columns_to_check].astype(float)
for column_name in columns_to_check:
    df_concatenated = eda.drop_outliers_zscore(df_concatenated, column_name, z_threshold=3)
df_concatenated.head(5)

In [10]:
df_concatenated.to_csv('/home/jabez/Documents/week_8/Logistic-optimization-Delivery-drivers-location-optimisation-with-Causal-Inference-/data/df_concatenated.csv')